In [17]:
import os
import pandas as pd

project_root = r"C:\Users\Diya\OneDrive\Desktop\quant-trading-system"
features_path = os.path.join(project_root, "data", "features", "nifty_features_spot_5min.csv")

df = pd.read_csv(features_path, parse_dates=["Date"])

# Ensure regime exists
print(df.columns)


Index(['Date', 'open', 'high', 'low', 'close', 'EMA_50', 'EMA_200', 'EMA_5',
       'EMA_15', 'returns', 'regime_label'],
      dtype='object')


In [19]:
import os
import pandas as pd
import numpy as np

project_root = r"C:\Users\Diya\OneDrive\Desktop\quant-trading-system"
features_path = os.path.join(project_root, "data", "features", "nifty_features_spot_5min.csv")

df = pd.read_csv(features_path, parse_dates=["Date"])

# -------------------------
# Strategy Signals
# -------------------------
df["signal"] = 0

# EMA crossover
df.loc[(df["EMA_5"] > df["EMA_15"]) & (df["EMA_5"].shift(1) <= df["EMA_15"].shift(1)), "signal"] = 1
df.loc[(df["EMA_5"] < df["EMA_15"]) & (df["EMA_5"].shift(1) >= df["EMA_15"].shift(1)), "signal"] = -1

# Apply regime filter
df.loc[df["regime_label"] == 0, "signal"] = 0

# -------------------------
# Backtesting
# -------------------------
df["position"] = df["signal"].replace({-1: -1, 1: 1}).ffill().fillna(0)

# Calculate returns
df["returns"] = df["close"].pct_change().fillna(0)

# Strategy returns
df["strategy_returns"] = df["returns"] * df["position"].shift(1)

# Cumulative returns
df["cumulative_returns"] = (1 + df["strategy_returns"]).cumprod()
df["cumulative_market"] = (1 + df["returns"]).cumprod()

# Save
output_path = os.path.join(project_root, "data", "signals")
os.makedirs(output_path, exist_ok=True)
df.to_csv(os.path.join(output_path, "nifty_signals_spot_5min.csv"), index=False)

print("✅ Baseline strategy completed & saved.")
print("Total trades:", len(df[df["signal"] != 0]))
print("Net return:", round((df["cumulative_returns"].iloc[-1] - 1) * 100, 2), "%")


✅ Baseline strategy completed & saved.
Total trades: 210
Net return: -0.43 %
